# Classes and Methods in Python

This notebook demonstrates:
- Instance variables and how they work across methods
- Class variables vs instance variables
- Common pitfalls with instance variables defined in methods

## Example 1: Instance Variables Across Methods (The Problem)

**Problem:** When you define an instance variable inside a method (not `__init__`), it only exists AFTER that method is called.

In [ ]:
# PROBLEM DEMO: Instance variable defined in a method, not in __init__

class EducationalInstitution:
    def __init__(self, name, location):
        # These are created when object is made - SAFE to use anywhere
        self.name = name
        self.location = location

    def display_info(self):
        print(self.name)
        self.another_method()  # Calls another_method which creates self.username
        print(self.username)   # Works because another_method() was called first
        
    def another_method(self):
        # WARNING: self.username is created HERE, not in __init__
        # It won't exist until this method is called!
        self.username = "admin"

    def yet_another_method(self):
        # DANGER: Tries to use self.username, but it may not exist yet!
        print(self.username)

In [ ]:
# Create an instance - only name and location exist at this point
EI = EducationalInstitution("ABC University", "New York")
# At this point: self.name ✓, self.location ✓, self.username ✗ (doesn't exist yet!)

In [ ]:
# ERROR! self.username doesn't exist because another_method() was never called
# This raises AttributeError: 'EducationalInstitution' object has no attribute 'username'
EI.yet_another_method()

AttributeError: 'EducationalInstitution' object has no attribute 'username'

## Example 2: Class Variables vs Instance Variables

**Class variables** are shared by ALL instances. **Instance variables** are unique to each object.

In [ ]:
# CLASS VARIABLES vs INSTANCE VARIABLES

class EducationalInstitution:
    # CLASS VARIABLES - defined outside __init__, shared by ALL instances
    university_name = "ABC University"
    university_location = "India"

    def __init__(self, college_name, college_location):
        # INSTANCE VARIABLES - unique to each object (use self.)
        self.college_name = college_name
        self.college_location = college_location
        
        # Accessing instance variables (use self.)
        print(f"College: {self.college_name}")
        print(f"Location: {self.college_location}")
        
        # Accessing class variables (use ClassName.)
        print(f"University: {EducationalInstitution.university_name}")
        print(f"University Location: {EducationalInstitution.university_location}")

In [ ]:
# Creating an instance - constructor runs and prints all info
EI = EducationalInstitution("ABC College", "Tamil Nadu")
# Class vars (university_name, university_location) are SAME for all objects
# Instance vars (college_name, college_location) are UNIQUE to this object

ABC College
Tamil Nadu
ABC University
india


## Example 3: Calling Methods from Constructor

You can call other methods from `__init__` to initialize more data.

In [ ]:
# CALLING A METHOD FROM __init__ - ensures instance variables are created early

class EducationalInstitution:
    university_name = "ABC University"  # Class variable
    university_location = "India"       # Class variable

    def __init__(self, college_name, college_location):
        self.college_name = college_name
        self.college_location = college_location
        
        # Call method from constructor - this creates principal_name & established_year
        # Now these variables exist immediately when object is created!
        info = self.display_college_info()
        print(f"Additional info: {info}")

    def display_college_info(self):
        # These instance variables are created when this method runs
        self.principal_name = "Dr. Smith"
        self.established_year = 1990
        return (self.principal_name, self.established_year)

In [ ]:
# Now when we create object, display_college_info() runs automatically
# So principal_name and established_year are available right away!
EI = EducationalInstitution("ABC College", "Tamil Nadu")

ABC College
Tamil Nadu
ABC University
india
('Dr. Smith', 1990)


## Example 4: Best Practice - Initialize All Variables in `__init__`

In [ ]:
# BEST PRACTICE: Initialize ALL instance variables in __init__

class EducationalInstitution:
    university_name = "ABC University"  # Class variable
    university_location = "India"       # Class variable

    def __init__(self, college_name, college_location):
        # Initialize ALL instance variables here - even if empty/None
        self.college_name = college_name
        self.college_location = college_location
        self.principal_name = None      # Will be set later
        self.established_year = None    # Will be set later

    def display_college_info(self):
        # Now we just UPDATE the variables, not create them
        self.principal_name = "Dr. Smith"
        self.established_year = 1990
        return (self.principal_name, self.established_year)

In [ ]:
# Safe! All variables exist from the start
EI = EducationalInstitution("ABC College", "Tamil Nadu")
print(f"Principal: {EI.principal_name}")  # None (but no error!)
EI.display_college_info()
print(f"Principal: {EI.principal_name}")  # "Dr. Smith"

ABC College
Tamil Nadu
ABC University
india
('Dr. Smith', 1990)
Dr. Smith
1990


---
## Real-World Example: Login System

A practical example showing class variables, instance variables, and methods working together.

In [ ]:
# LOGIN SYSTEM - Version 1 (has a bug!)

class LoginSystem:
    # CLASS VARIABLE - same limit for ALL users
    max_login_attempts = 5

    def __init__(self, username, password):
        # INSTANCE VARIABLES - unique to each user
        self.username = username
        self.password = password
        self.is_logged_in = False
        # BUG: login_attempts not initialized here!
        print(f"User {self.username} registered successfully!")

    def login(self, username, password):
        if self.username == username and self.password == password:
            self.is_logged_in = True
            self.login_attempts = 0  # Created only on SUCCESS
            print("Login successful!")
            return True
        else:
            # getattr() used because login_attempts might not exist yet
            self.login_attempts = getattr(self, 'login_attempts', 0) + 1
            print(f"Login failed! Attempt {self.login_attempts}")
            return False

In [ ]:
# Create two users
ls = LoginSystem("user1", "pass123")
ls1 = LoginSystem("user2", "pass456")

User user1 registered successfully!


In [ ]:
# Successful login
ls.login("user1", "pass123")

Login successful!


True

### Login System - Improved Version (Fixed)

In [ ]:
# LOGIN SYSTEM - Version 2 (FIXED - all variables in __init__)

class LoginSystem:
    # CLASS VARIABLE - shared by all instances (same limit for everyone)
    max_login_attempts = 5

    def __init__(self, username, password):
        # ALL INSTANCE VARIABLES initialized here - BEST PRACTICE!
        self.username = username
        self.password = password
        self.is_logged_in = False
        self.login_attempts = 0  # ✓ Now initialized properly!
        print(f"User {self.username} registered successfully!")

    def login(self, username, password):
        # Check if account is locked (using CLASS variable)
        if self.login_attempts >= LoginSystem.max_login_attempts:
            print(f"Account locked! Max attempts ({LoginSystem.max_login_attempts}) exceeded.")
            return False
        
        # Check credentials
        if self.username == username and self.password == password:
            self.is_logged_in = True
            self.login_attempts = 0  # Reset on success
            print("Login successful!")
            return True
        else:
            self.login_attempts += 1  # Safe! Variable exists
            remaining = LoginSystem.max_login_attempts - self.login_attempts
            print(f"Login failed! Attempt {self.login_attempts}/{LoginSystem.max_login_attempts}. Remaining: {remaining}")
            return False

    def logout(self):
        if self.is_logged_in:
            self.is_logged_in = False
            print(f"User {self.username} logged out")
        else:
            print("No user logged in")

In [ ]:
# Create a user and try wrong password
ls01 = LoginSystem("user1", "pass123")
ls01.login("user1", "wrongpass")  # Attempt 1

User user1 registered successfully!
Login failed! Attempt 1/5. Remaining attempts: 4


False

In [ ]:
# Correct password - resets attempt counter
ls01.login("user1", "pass123")

Login successful!


True

In [ ]:
# Wrong password again - attempt 1 (counter was reset)
ls01.login("user1", "wrongpass")

Login failed! Attempt 3/5. Remaining attempts: 2


False

In [ ]:
# Attempt 2
ls01.login("user1", "wrongpass")

Login failed! Attempt 4/5. Remaining attempts: 1


False

In [ ]:
# Attempt 3
ls01.login("user1", "wrongpass")

Login failed! Attempt 5/5. Remaining attempts: 0


False

In [ ]:
# Attempt 4
ls01.login("user1", "wrongpass")

Maximum login attempts (5) exceeded. Account locked.


False

In [ ]:
# Attempt 5 - this will be the last allowed attempt
ls01.login("user1", "wrongpass")

In [ ]:
# Attempt 6 - Account should be locked now!
ls01.login("user1", "pass123")  # Even correct password won't work

### Key Point: Class Variable is Shared

In [ ]:
# CLASS VARIABLE vs INSTANCE VARIABLE demonstration

# Create a new user - has its own login_attempts (instance var)
ls02 = LoginSystem("user2", "pass456")

# Class variable - same for ALL users
print(f"Max attempts (class var): {LoginSystem.max_login_attempts}")

# Instance variable - unique to each user
print(f"ls01 attempts (instance var): {ls01.login_attempts}")
print(f"ls02 attempts (instance var): {ls02.login_attempts}")

# Changing class variable affects ALL instances
LoginSystem.max_login_attempts = 3
print(f"\nAfter changing class var to 3:")
print(f"ls01 max: {ls01.max_login_attempts}")  # Both see the new value
print(f"ls02 max: {ls02.max_login_attempts}")

0